In [1]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm # to create loadbard in for loop
import os
from datetime import date
import pickle

## API data response

<pre>
dict( 
    data : list(one element is one earning call metada)
    included : [first element don't care,{get id[id],get attributes{name:ticker}}] # corresponding data in order 
    meta : "don't care for now"
        )
</pre>
One earning call metadata is show below 

In [2]:
# need to precise the option for date, nbr of metada for call transcript in the url paramater 
# https://en.wikipedia.org/wiki/Query_string
nbr_earning_call_metada = 1500
date_of_request = date.today().strftime("%Y-%m-%d") #e.g. "2022-04-28"
max_return_per_request = 50
api_data_responses = []

data_path = "data/id_to_ticker"
if not os.path.exists(data_path):
    id_mapping = {}
else:
    with open(data_path,"rb") as file :
        id_mapping = pickle.load(file)

for page_number in range(1,nbr_earning_call_metada//max_return_per_request + 1):
    url = "https://seekingalpha.com/api/v3/articles"+"?cacheBuster="\
          +str(date_of_request)\
          +"&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0"\
          +"&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]="\
          +str(max_return_per_request)\
          +"&page[number]="+str(page_number)
    # the chrome driver depend on which version of chrome you are running specify the good one
    executable_path = "./chromedriver"
    service = Service(executable_path)
    option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service,options=option)
    driver.get(url)
    api_json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
    api_data_responses.extend(json.loads(api_json_response)["data"])
    api_included_responses = json.loads(api_json_response)["included"]
    dict_included = {}
    
    for element in api_included_responses:
        dict_included[element["id"]] = element["attributes"]    
    
    for i in range(max_return_per_request):
        ticker_id = api_data_responses[-50+i]
        ticker_id = ticker_id["relationships"]["primaryTickers"]["data"][0]["id"]
        if ticker_id not in dict_included:
            raise ValueError("The ticker id is not in the dictionnary included")
        ticker_included = dict_included[ticker_id]["name"]
        id_mapping[api_data_responses[-50+i]["id"]] = ticker_included 
        
with open(data_path,"wb") as file : 
    pickle.dump(id_mapping, file, protocol=pickle.HIGHEST_PROTOCOL)
print(url)
#a quick loock of the result to check if we get results are get captcha/bot error 
print(api_json_response[0:100])

KeyError: 'data'

In [ ]:
with open(data_path,"rb") as file :
    id_mapping = pickle.load(file)
    print(len(id_mapping))

In [ ]:
api_data_responses[0]

In [ ]:
id_and_path_to_earning_calls = [ (earning_call_metada["id"] ,earning_call_metada["links"]["self"]) 
                     for earning_call_metada in api_data_responses]
# ""check url
id_and_path_to_earning_calls[0:2]

In [ ]:
data_path = "data/date"
if not os.path.exists(data_path):
    date = {}
else:
    with open(data_path,"rb") as file :
        date = pickle.load(file)
for elem in api_data_responses:
    date[elem["id"]] = elem["attributes"]["publishOn"][:10]
    
with open(data_path,"wb") as file : 
    pickle.dump(date, file, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(data_path,"rb") as file :
    date = pickle.load(file)
    print(date)
    print(len(date))

## download all not yet download html from url

In [ ]:
domain_name = 'https://seekingalpha.com'
data_path = "data/article"
need_to_download = []
if not os.path.exists(data_path):
    os.makedirs(data_path)
    print("The new directory is created!")
for id_earning_call, path_to_earning_call in tqdm(id_and_path_to_earning_calls):
    if not os.path.isfile(data_path+"/"+ id_earning_call +".html"):
        # not in local directory
        need_to_download.append((id_earning_call, path_to_earning_call))
print(f"Need to download {len(need_to_download)} html file(s)")
for id_earning_call, path_to_earning_call in tqdm(need_to_download):
    with open(data_path+"/"+ id_earning_call+".html", "x") as file:
            url = domain_name + path_to_earning_call
            html_transcript = requests.get(url).text
            file.write(html_transcript)
    

## Check last file got downloaded, i.e. captcha/bot error 

In [ ]:
BeautifulSoup(html_transcript).get_text()[:2000]